## Hent data fra Bibsys autoritetsregister

In [1]:
import requests

def get_author(author):
    query = {'q': author,
             'start':1,
             'max':10,
             'format':'json'}
    res = requests.get("https://authority.bibsys.no/authority/rest/functions/v2/query", params = query)
    return res.json()

def get_author_info_from_key(key):
    query = {'format':'json'}
    res = requests.get("https://authority.bibsys.no/authority/rest/authorities/v2/{key}".format(key=key), params = query)
    return res.json()

def get_marc_post(name):
    r = get_author(name)

    results = r['results']
    #print(results)
    identifiers = [x['identifiersMap']['autid'][0] for x in results if 'autid' in x['identifiersMap']] 
    #print(identifier)
    res = []
    try:
        for identifier in identifiers:
            identifier = identifier.split('x')[1]
            try:
                #print(identifier)
                res.append(get_author_info_from_key(identifier))
            except:
                True #print('feil med:', identifier)
    except:
        True
    return res

def posts(name):
    return [x for x in get_marc_post(name) if x['authorityType'] == 'PERSON']

def life_span(marcpost):
    return [date['value'] for item in  marcpost[0]['marcdata'] for date in item['subfields'] if date['subcode'] == 'd' and item['tag'] == '100']

def alternatives(marcpost):
    return [alt['value'] for item in  marcpost[0]['marcdata'] for alt in item['subfields'] if alt['subcode'] == 'a' and item['tag'] == '400']

def get_authority_info(name):
    marcpost = posts(name)
    if marcpost != []:
        lf = life_span(marcpost)
        alter = alternatives(marcpost)
    else:
        lf = []
        alter = []
    return {'life-span': lf, 'alternatives': alter}

In [6]:
import re
import pandas as pd

In [7]:
import dhlab.nbtext as nb

In [18]:
df = pd.read_json('authority.json')

In [29]:
df.death = df.death.apply(lambda x:x[:4])

TypeError: 'NoneType' object is not subscriptable

In [51]:
df[(df.birth < "1880") & (df.death > "1800")].to_excel('forfattere_ff_1880.xls')

In [52]:
df[(df.birth < "1880") & (df.death > "1800")]

,lifespan,birth,death
Henrik Wergeland,1808-1845,1808,1845
Henrik Ibsen,1828-1906,1828,1906
Jonas Lie,1868-1943,1868,1943
Camilla Collett,1813-1895,1813,1895
Arne Garborg,1851-1924,1851,1924
...,...,...,...
Nordahl Brun Rolfsen,1848-1928,1848,1928
Johan Christian Heuch,1838-1904,1838,1904
Gustav Peter Blom,1785-1869,1785,1869
Kristian Oskar Bjørnstad,1875-1968,1875,1968
